<a href="https://colab.research.google.com/github/Djensonsan/Information_Retrieval_Assignment_2/blob/main/LDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Information Retrieval Assignment 2: LDA


## Runtime specs

In [ ]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 85
model name	: Intel(R) Xeon(R) CPU @ 2.00GHz
stepping	: 3
microcode	: 0x1
cpu MHz		: 2000.186
cache size	: 39424 KB
physical id	: 0
siblings	: 4
core id		: 0
cpu cores	: 2
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm mpx avx512f avx512dq rdseed adx smap clflushopt clwb avx512cd avx512bw avx512vl xsaveopt xsavec xgetbv1 xsaves arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs taa
bogomips	:

In [ ]:
!cat /proc/meminfo

MemTotal:       26751732 kB
MemFree:        21632444 kB
MemAvailable:   25739404 kB
Buffers:           99444 kB
Cached:          4197412 kB
SwapCached:            0 kB
Active:          1356236 kB
Inactive:        3388708 kB
Active(anon):     405960 kB
Inactive(anon):      404 kB
Active(file):     950276 kB
Inactive(file):  3388304 kB
Unevictable:           0 kB
Mlocked:               0 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:              1012 kB
Writeback:             0 kB
AnonPages:        448076 kB
Mapped:           242644 kB
Shmem:               940 kB
Slab:             240084 kB
SReclaimable:     182336 kB
SUnreclaim:        57748 kB
KernelStack:        4976 kB
PageTables:         6200 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:    13375864 kB
Committed_AS:    3220564 kB
VmallocTotal:   34359738367 kB
VmallocUsed:           0 kB
VmallocChunk:          0 kB
Percpu:             1904 kB
AnonHugePages:   

## Imports

In [1]:
# Install your required packages here
!pip install pandas numpy matplotlib fsspec gcsfs dask
!pip install -q tqdm

     |████████████████████████████████| 92kB 3.5MB/s 
     |████████████████████████████████| 1.3MB 6.7MB/s 
     |████████████████████████████████| 296kB 15.6MB/s 
     |████████████████████████████████| 143kB 15.9MB/s 
  Created wheel for idna-ssl: filename=idna_ssl-1.1.0-cp36-none-any.whl size=3161 sha256=908dce32054433178ee26d5af8f86df1f855e932cbee45d8162867f1c4f5f482
  Stored in directory: /root/.cache/pip/wheels/d3/00/b3/32d613e19e08a739751dd6bf998cfed277728f8b2127ad4eb7
Successfully built idna-ssl


In [47]:
import pandas as pd
import numpy as np
import tqdm.notebook as tqdm
import dask.dataframe as dd

from ast import literal_eval
import gensim
from gensim import corpora, models
from tqdm import tqdm
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from nltk.corpus import stopwords
import nltk
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
# Mount google drive in colab:
from google.cloud import storage
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Preprocessing

In [ ]:
# Pycharm:
# data = pd.read_csv('data/news_dataset.csv')

In [56]:
# Colab:
data = pd.read_csv('/content/drive/MyDrive/IR-Assignment-2/data/news_dataset.csv')

### Exploration

In [57]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141585 entries, 0 to 141584
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   id           141585 non-null  int64 
 1   title        141583 non-null  object
 2   publication  141564 non-null  object
 3   author       125725 non-null  object
 4   date         138869 non-null  object
 5   year         138911 non-null  object
 6   month        138911 non-null  object
 7   url          84541 non-null   object
 8   content      141543 non-null  object
dtypes: int64(1), object(8)
memory usage: 9.7+ MB


In [72]:
data.head(n=43)

,id,title,publication,author,date,year,month,url,content
0,1,House Republicans Fret About Winning Their Hea...,New York Times,Carl Hulse,2016-12-31,2016,12,NaN,WASHINGTON — Congressional Republicans have...
1,2,Rift Between Officers and Residents as Killing...,New York Times,Benjamin Mueller and Al Baker,2017-06-19,2017,6,NaN,"After the bullet shells get counted, the blood..."
2,3,"Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...",New York Times,Margalit Fox,2017-01-06,2017,1,NaN,"When Walt Disney’s “Bambi” opened in 1942, cri..."
3,4,"Among Deaths in 2016, a Heavy Toll in Pop Musi...",New York Times,William McDonald,2017-04-10,2017,4,NaN,"Death may be the great equalizer, but it isn’t..."
4,5,Kim Jong-un Says North Korea Is Preparing to T...,New York Times,Choe Sang-Hun,2017-01-02,2017,1,NaN,"SEOUL, South Korea — North Korea’s leader, ..."
5,6,"Sick With a Cold, Queen Elizabeth Misses New Y...",New York Times,Sewell Chan,2017-01-02,2017,1,NaN,"LONDON — Queen Elizabeth II, who has been b..."
6,7,Taiwan’s President Accuses China of Renewed In...,New York Times,Javier C. Hernández,2017-01-02,2017,1,NaN,BEIJING — President Tsai of Taiwan sharpl...
7,8,"After ‘The Biggest Loser,’ Their Bodies Fought...",New York Times,Gina Kolata,2017-02-08,2017,2,NaN,"Danny Cahill stood, slightly dazed, in a blizz..."
8,9,"First, a Mixtape. Then a Romance. - The New Yo...",New York Times,Katherine Rosman,2016-12-31,2016,12,NaN,"Just how is Hillary Kerr, the founder of ..."
9,10,Calling on Angels While Enduring the Trials of...,New York Times,Andy Newman,2016-12-31,2016,12,NaN,Angels are everywhere in the Muñiz family’s ap...


In [67]:
# Use document 42 as running example
data.loc[42, 'content']

'WASHINGTON  —     Donald J. Trump on Tuesday named as his chief trade negotiator a Washington lawyer who has long advocated protectionist policies, the latest sign that Mr. Trump intends to fulfill his campaign promise to get tough with China, Mexico and other trading partners. Mr. Trump also renewed his episodic campaign to persuade American companies to expand domestic manufacturing, criticizing General Motors via Twitter on Tuesday morning for making in Mexico some of the Chevrolet Cruze hatchbacks it sells domestically. Hours later, Mr. Trump claimed credit after Ford said it would expand vehicle production in Flat Rock, Mich. The choice of Robert Lighthizer (pronounced   ) to be the United States’ trade representative nearly completes Mr. Trump’s selection of top economic advisers and, taken together with the  ’s running commentary on Twitter, underscores Mr. Trump’s focus on making things in America. That is causing unease among some Republicans who regard Mr. Trump’s views on t

### Keep document content

In [69]:
data_content = data['content']

In [70]:
type(data_content)

pandas.core.series.Series

In [71]:
data_content.head(n=3)

0    WASHINGTON  —   Congressional Republicans have...
1    After the bullet shells get counted, the blood...
2    When Walt Disney’s “Bambi” opened in 1942, cri...
Name: content, dtype: object

### Tokenization, Stemming and Lemmatization

In [ ]:
tqdm.pandas()
# Note Jens: Might want to use Dask to speed things up. 
# When using Dask can't use tqdm as far as I know.

In [ ]:
# There's NaN values in the dataset
data_content.dropna(inplace=True)

In [ ]:
data_content.isna().any()

False

In [ ]:
# Tokenization
data_content_tokenized = data_content.progress_apply(lambda x: nltk.word_tokenize(x))

100%|██████████| 141543/141543 [14:52<00:00, 158.56it/s]


In [ ]:
# Remove words smaller than 3 characters
data_content_tokenized = data_content_tokenized.progress_apply(lambda x: [y for y in x if len(y)>2])

100%|██████████| 141543/141543 [00:19<00:00, 7158.94it/s]


In [ ]:
# Stemming and Lemmatization 
stemmer = SnowballStemmer("english")
data_content_stemmed = data_content_tokenized.progress_apply(lambda x: [stemmer.stem(WordNetLemmatizer().lemmatize(y)) for y in x])

100%|██████████| 141543/141543 [25:06<00:00, 93.98it/s]


In [ ]:
# Remove Stopswords
stop_words = set(stopwords.words('english')) 
data_content_clean = data_content_stemmed.progress_apply(lambda x: [y for y in x if not y in stop_words])

100%|██████████| 141543/141543 [00:16<00:00, 8671.00it/s]


In [ ]:
data_content_clean.head()

0    [washington, congression, republican, new, fea...
1    [bullet, shell, get, count, blood, dri, votiv,...
2    [walt, disney, bambi, open, 1942, critic, prai...
3    [death, may, great, equal, necessarili, evenha...
4    [seoul, south, korea, north, korea, leader, ki...
Name: content, dtype: object

In [ ]:
# data_content_clean contain the cleaned 'content' column of the news dataset:
data_content_clean.to_csv('/content/drive/MyDrive/IR-Assignment-2/data/new_dataset_clean.csv')

## LDA Library

The following part will use Python libraries for performing an LDA. The result can be used for comparison with our model. 

The code follows a tutorial:
[LDA Tutorial](https://towardsdatascience.com/topic-modeling-and-latent-dirichlet-allocation-in-python-9bf156893c24) 

In [73]:
data_content_clean = pd.read_csv('/content/drive/MyDrive/IR-Assignment-2/data/new_dataset_clean.csv')

In [74]:
data_content_clean = data_content_clean['content']

In [75]:
data_content_clean.head()

0    ['washington', 'congression', 'republican', 'n...
1    ['bullet', 'shell', 'get', 'count', 'blood', '...
2    ['walt', 'disney', 'bambi', 'open', '1942', 'c...
3    ['death', 'may', 'great', 'equal', 'necessaril...
4    ['seoul', 'south', 'korea', 'north', 'korea', ...
Name: content, dtype: object

In [76]:
dictionary = gensim.corpora.Dictionary()
for article in tqdm(data_content_clean):
  dictionary.add_documents([literal_eval(article)])

100%|██████████| 141543/141543 [03:42<00:00, 635.75it/s]


## Sanity check

In [77]:
print(dictionary)

Dictionary(274259 unique tokens: ['2010', '2015', '2017', 'access', 'acknowledg']...)


In [78]:
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 2010
1 2015
2 2017
3 access
4 acknowledg
5 act
6 administr
7 advoc
8 afford
9 alli
10 american


In [81]:
# Filter out tokens that appear in less than 15 documents, more than 0.5 documents (fraction of total corpus size, not absolute number).
# keep only the first 100000 most frequent tokens.
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

## doc2bow

For each document we create a dictionary reporting how many words and how many times those words appear.

In [82]:
bow_corpus = [dictionary.doc2bow(literal_eval(doc)) for doc in data_content_clean]
bow_corpus[42]

[(1, 1),
 (3, 1),
 (6, 5),
 (7, 3),
 (8, 1),
 (10, 10),
 (13, 1),
 (26, 1),
 (35, 2),
 (36, 1),
 (47, 1),
 (52, 1),
 (53, 2),
 (55, 1),
 (58, 1),
 (60, 1),
 (62, 1),
 (63, 1),
 (67, 1),
 (71, 1),
 (74, 1),
 (78, 1),
 (80, 1),
 (85, 1),
 (99, 2),
 (101, 2),
 (115, 3),
 (117, 1),
 (124, 2),
 (128, 6),
 (136, 2),
 (138, 1),
 (139, 2),
 (141, 1),
 (144, 1),
 (145, 1),
 (147, 1),
 (148, 2),
 (150, 3),
 (151, 2),
 (154, 1),
 (158, 1),
 (162, 1),
 (163, 1),
 (165, 1),
 (170, 1),
 (171, 32),
 (172, 3),
 (175, 1),
 (176, 2),
 (184, 1),
 (201, 1),
 (202, 1),
 (205, 1),
 (212, 1),
 (216, 1),
 (219, 3),
 (225, 5),
 (227, 1),
 (231, 1),
 (232, 3),
 (234, 2),
 (235, 2),
 (236, 1),
 (238, 1),
 (241, 1),
 (247, 5),
 (251, 1),
 (259, 3),
 (266, 1),
 (270, 23),
 (274, 4),
 (277, 2),
 (278, 2),
 (281, 1),
 (282, 3),
 (283, 1),
 (285, 2),
 (313, 1),
 (314, 1),
 (319, 4),
 (332, 1),
 (333, 1),
 (334, 1),
 (338, 2),
 (348, 1),
 (374, 1),
 (376, 1),
 (403, 1),
 (411, 2),
 (427, 2),
 (429, 2),
 (443, 3),
 (44

In [83]:
bow_doc_42 = bow_corpus[42]
for i in range(len(bow_doc_42)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_42[i][0], 
                                               dictionary[bow_doc_42[i][0]], 
bow_doc_42[i][1]))

Word 1 ("2015") appears 1 time.
Word 3 ("access") appears 1 time.
Word 6 ("administr") appears 5 time.
Word 7 ("advoc") appears 3 time.
Word 8 ("afford") appears 1 time.
Word 10 ("american") appears 10 time.
Word 13 ("anoth") appears 1 time.
Word 26 ("big") appears 1 time.
Word 35 ("case") appears 2 time.
Word 36 ("caus") appears 1 time.
Word 47 ("committe") appears 1 time.
Word 52 ("congress") appears 1 time.
Word 53 ("congression") appears 2 time.
Word 55 ("conserv") appears 1 time.
Word 58 ("consum") appears 1 time.
Word 60 ("continu") appears 1 time.
Word 62 ("cost") appears 1 time.
Word 63 ("could") appears 1 time.
Word 67 ("deal") appears 1 time.
Word 71 ("demand") appears 1 time.
Word 74 ("despit") appears 1 time.
Word 78 ("discuss") appears 1 time.
Word 80 ("disput") appears 1 time.
Word 85 ("donald") appears 1 time.
Word 99 ("even") appears 2 time.
Word 101 ("execut") appears 2 time.
Word 115 ("general") appears 3 time.
Word 117 ("govern") appears 1 time.
Word 124 ("hous") app

## TF-IDF

In [49]:
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
for doc in corpus_tfidf:
    print(doc)
    break

[(0, 0.029715521149501007), (1, 0.020106039956814977), (2, 0.027461976654113714), (3, 0.024622054606840662), (4, 0.027714806504477944), (5, 0.01740509473557033), (6, 0.22367532184254252), (7, 0.0293810665214199), (8, 0.02987825602451966), (9, 0.02641312959161405), (10, 0.011392676082327427), (11, 0.03410362327267204), (12, 0.029880923564519137), (13, 0.012355589545940467), (14, 0.07762049647147408), (15, 0.1068024907136795), (16, 0.09967311773417871), (17, 0.026996404886106364), (18, 0.03652412412495001), (19, 0.03442674718769445), (20, 0.03473520417223675), (21, 0.025944516344889656), (22, 0.04378686846569487), (23, 0.04116666507506936), (24, 0.009390811128090697), (25, 0.036116844179606186), (26, 0.017040391324804997), (27, 0.04647128590014654), (28, 0.09261272437117676), (29, 0.058799551481910003), (30, 0.1872871811821654), (31, 0.03159758088838345), (32, 0.03758666316511679), (33, 0.14850272962818895), (34, 0.06034014240082647), (35, 0.05978056802508807), (36, 0.0391396559532639), 

## LDA

In [84]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=20, id2word=dictionary, passes=2, workers=2)

In [85]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.029*"trump" + 0.017*"republican" + 0.014*"vote" + 0.014*"democrat" + 0.014*"clinton" + 0.012*"campaign" + 0.012*"elect" + 0.012*"voter" + 0.011*"parti" + 0.010*"state"
Topic: 1 
Words: 0.024*"polic" + 0.013*"offic" + 0.011*"kill" + 0.009*"attack" + 0.008*"report" + 0.007*"death" + 0.007*"shoot" + 0.007*"told" + 0.006*"two" + 0.006*"prison"
Topic: 2 
Words: 0.011*"citi" + 0.005*"home" + 0.005*"area" + 0.005*"water" + 0.004*"park" + 0.004*"car" + 0.004*"build" + 0.004*"day" + 0.004*"two" + 0.004*"000"
Topic: 3 
Words: 0.027*"court" + 0.015*"law" + 0.012*"rule" + 0.011*"case" + 0.010*"state" + 0.010*"judg" + 0.009*"justic" + 0.009*"right" + 0.009*"suprem" + 0.008*"feder"
Topic: 4 
Words: 0.018*"china" + 0.013*"russia" + 0.013*"russian" + 0.010*"bank" + 0.010*"putin" + 0.008*"chines" + 0.008*"govern" + 0.006*"presid" + 0.005*"report" + 0.005*"foreign"
Topic: 5 
Words: 0.020*"percent" + 0.009*"tax" + 0.008*"market" + 0.008*"rate" + 0.007*"billion" + 0.006*"job" + 0.006*"t

## Evaluation

In [86]:
# Let's look at a test document: document 42
# 1) clearly see that the article is about the economy 40%
# 2) US politics 27%
# 3) US Justice Department 8%
# 4) US Congress 6%
for index, score in sorted(lda_model[bow_corpus[42]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.4059540033340454	 
Topic: 0.020*"percent" + 0.009*"tax" + 0.008*"market" + 0.008*"rate" + 0.007*"billion" + 0.006*"job" + 0.006*"trade" + 0.006*"econom" + 0.006*"price" + 0.006*"economi"

Score: 0.26900607347488403	 
Topic: 0.060*"trump" + 0.018*"clinton" + 0.016*"presid" + 0.011*"obama" + 0.008*"polit" + 0.008*"donald" + 0.007*"campaign" + 0.006*"american" + 0.006*"hillari" + 0.005*"polici"

Score: 0.07334913313388824	 
Topic: 0.027*"court" + 0.015*"law" + 0.012*"rule" + 0.011*"case" + 0.010*"state" + 0.010*"judg" + 0.009*"justic" + 0.009*"right" + 0.009*"suprem" + 0.008*"feder"

Score: 0.06897629052400589	 
Topic: 0.022*"hous" + 0.019*"republican" + 0.017*"north" + 0.015*"senat" + 0.015*"korea" + 0.012*"ryan" + 0.009*"presid" + 0.009*"democrat" + 0.009*"congress" + 0.008*"bill"

Score: 0.062184739857912064	 
Topic: 0.018*"china" + 0.013*"russia" + 0.013*"russian" + 0.010*"bank" + 0.010*"putin" + 0.008*"chines" + 0.008*"govern" + 0.006*"presid" + 0.005*"report" + 0.005*"fore